<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32b_langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now we will begin exploring Conversational Memory for LLMs using Langchain.


# 1. Import Library

Install OpenAI and LangChain Library by Uncomment Cell below and Run (Shift + Enter)

In [7]:
# ! pip install langchain
# !pip install openai


Import the LangChain Library

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 2. Authentication

Before we can access and utilize ChatGPT and GPT-4 from OpenAI, we need to register, sign up, and provide the necessary payment information.

## Sign up

Click on the green Sign up button in the link below to register for OpenAI.

https://platform.openai.com/overview

Please enter your credit card information on the Billing page.

https://platform.openai.com/account/billing/payment-methods






## API Key

To generate an API Key, please follow the steps below:

1. Access the link provided below.
2. Click on the "Create new secret key" button.
3. Give a name to the key.
4. Copy the secret key generated and keep it ready to be entered in the "OPENAI_API_KEY" field below.

Link: https://platform.openai.com/account/api-keys

Please make sure to securely store and protect your API Key as it grants access to the OpenAI API.


## Sample API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

Please execute the cell below and enter the secret key.


In [4]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


After completing the testing, it is recommended to go back and delete the Secret Key on the page https://platform.openai.com/account/api-keys every time for security purposes. We can generate new keys continuously and without limitations.

#3. LangChain Memory

## 3.1 ConversationBufferMemory

The ConversationBufferMemory is a tool that can save messages and then retrieve them using a variable.

We will initialize ChatOpenAI with temperature=0.0 to make it repeatable. Please note that verbose=True.

In [12]:
chatllm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chatllm, 
    memory = memory,
    verbose=True
)

First, we'll start by greeting each other to test how everything functions. You'll notice a prompt LangChain set up behind the scenes in green.

In [13]:
conversation.predict(input="Hello, my name is Ken")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, my name is Ken
AI:

> Finished chain.


"Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?"

Then a little arithmetic. 

You can see LangChain send all previous prompts to LLM.

In [14]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

Even if the language model is stateless, it can still use the information from the previous chat message to answer the question.

In [11]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ken, as you mentioned earlier.'

We can take a look inside the memory (chat log).

In [15]:
print(memory.buffer)

Human: Hello, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.


We could retrive all the history buffer.

In [16]:
memory.load_memory_variables({})

{'history': "Human: Hello, my name is Ken\nAI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2."}

### Craft new memory

We have the option to create new memories in any way we prefer.

In [21]:
memory = ConversationBufferMemory()

In [22]:
memory.save_context({"input": "Roses are blue."}, 
                    {"output": "Violets are red."})

In [23]:
print(memory.buffer)

Human: Roses are blue.
AI: Violets are red.


In [24]:
memory.load_memory_variables({})

{'history': 'Human: Roses are blue.\nAI: Violets are red.'}

Add more memory.

In [25]:
memory.save_context({"input": "A poetic twist"}, 
                    {"output": "from what's often said."})

In [26]:
memory.load_memory_variables({})

{'history': "Human: Roses are blue.\nAI: Violets are red.\nHuman: A poetic twist\nAI: from what's often said."}

## 3.2 ConversationBufferWindowMemory

ConversationBufferWindowMemory maintains a record of the conversation's interactions. The WINDOW only stores the most recent K interactions. This is helpful for keeping track of the latest interactions without letting the buffer become too large.

We will import ConversationBufferWindowMemory from memory package

In [30]:
from langchain.memory import ConversationBufferWindowMemory

Set the window size (k) to 1.

In [31]:
memory = ConversationBufferWindowMemory(k=1)      

In [32]:
memory.save_context({"input": "Roses are blue."}, 
                    {"output": "Violets are red."})
memory.save_context({"input": "A poetic twist"}, 
                    {"output": "from what's often said."})

The memory can only hold the most recent interaction.

In [33]:
memory.load_memory_variables({})

{'history': "Human: A poetic twist\nAI: from what's often said."}

We will try the case above.

In [34]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [35]:
conversation.predict(input="Hi, my name is Ken")

"Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?"

In [36]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [ ]:
# memory.load_memory_variables({})

The conversation memory doesn't contain a name, so the language model cannot respond to the question.

In [38]:
conversation.predict(input="What is my name?")

"I'm sorry, I still don't have access to that information. However, I can tell you that your IP address is 192.168.1.1 and you are currently using a Chrome browser version 91.0.4472.124 on a Windows 10 operating system. Is there anything else I can help you with?"

In [39]:
memory.load_memory_variables({})

{'history': "Human: What is my name?\nAI: I'm sorry, I still don't have access to that information. However, I can tell you that your IP address is 192.168.1.1 and you are currently using a Chrome browser version 91.0.4472.124 on a Windows 10 operating system. Is there anything else I can help you with?"}

## 3.3 ConversationTokenBufferMemory

The ConversationTokenBufferMemory stores recent interactions in memory. Instead of counting the number of interactions, it determines when to clear the interactions based on the length of tokens (words) used.

The main distinction of ConversationTokenBufferMemory is that it sets a limit based on the number of words in the stored messages. This is unlike ConversationBufferWindowMemory, which removes interactions based on the number of turns.

In [41]:
# ! pip install tiktoken

In [42]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

Set max_token_limit to 40

In [47]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)
memory.save_context({"input": "Doe"},
                    {"output": "A deer, a female deer."})
memory.save_context({"input": "Ray"},
                    {"output": "A drop of golden sun."})
memory.save_context({"input": "Me"}, 
                    {"output": "A name I call myself."})

In [48]:
memory.load_memory_variables({})

{'history': 'Human: Ray\nAI: A drop of golden sun.\nHuman: Me\nAI: A name I call myself.'}

Set max_token_limit to 60

In [49]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=60)
memory.save_context({"input": "Doe"},
                    {"output": "A deer, a female deer."})
memory.save_context({"input": "Ray"},
                    {"output": "A drop of golden sun."})
memory.save_context({"input": "Me"}, 
                    {"output": "A name I call myself."})

In [50]:
memory.load_memory_variables({})

{'history': 'Human: Doe\nAI: A deer, a female deer.\nHuman: Ray\nAI: A drop of golden sun.\nHuman: Me\nAI: A name I call myself.'}

## 3.4 ConversationSummaryMemory

ConversationSummaryMemory is a kind of memory that summarizes the conversation as time goes on. It helps to condense information from the conversation over time.

When the large language model runs, it includes the conversation history using these memory nodes. Instead of using the entire conversation history, we use ConversationSummaryMemory, which provides a summary of the conversation without losing important information. This helps to save space and stay within the token limit.

In [51]:
from langchain.memory import ConversationSummaryBufferMemory


In [37]:
# create a long string
news = """
A new six-story college campus building stands as Singapore’s latest ode to \
nature. Home to Nanyang Technological University’s (NTU) business school, the \
gently curved design features sunlit atriums, open-air study areas set against \
lush backdrops and elevators that descend into beds of tropical plants. \
Everything from handrails to benches, door frames to room dividers (and even \
                                                                    an adjoining bus stop), were built using wood.

So, too, were the structural beams and columns. In fact, the building is \
made almost entirely from mass timber — a new generation of engineered wood, \
arranged in layers and bonded with strong adhesives, that is pushing \
the boundaries of architecture. Sprawling across 43,500 square meters \
 (468,000 square feet), it is now Asia’s largest timber building, by floor area.

Named Gaia, after the ancient Greek goddess of the Earth, the project \
opened in May and cost 125 million Singapore dollars ($93 million) to build. \
Its exposed timber frame is free from cladding or paint, a design \
decision that celebrates natural materials while giving visitors \
the feeling of walking between trees.

According to the celebrated Japanese architect behind the project, \
Toyo Ito, this was precisely the point. “I always try to envision \
a connection with — and a feeling of — nature, such as trees and water, \
in my designs,” he told CNN shortly after the building’s inauguration \
ceremony. “The fact you mentioned that if feels like entering a forest \
shows that my vision came through.”
"""

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{news}"})

In [38]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [39]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [40]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a demo for the meeting?"